In [1]:
import pandas as pd

pd.options.display.max_columns = 500

In [2]:
df_train = pd.read_csv('train_data_nonull.csv')
df_val = pd.read_csv('val_data_nonull.csv')

In [3]:
keys = ['id', 'vintage']
product = ['installments']
relevance_features = ['is_loan', 'is_default', 'relevance_score']
hold_feature = ['internal_feature_3']

all_remaining = [col for col in df_train.columns if col not in keys + product + relevance_features + hold_feature]

Treatment for categorical features

In [4]:
def get_regiao(value):
    if value in ['RS', 'PR', 'SC']:
        new_value = 'Sul'
    elif value in ['MG', 'RJ', 'ES']:
        new_value = 'Sudeste'
    elif value in ['MS', 'MT', 'GO', 'DF']:
        new_value = 'Centro_Oeste'
    elif value in ['BA', 'PI', 'MA', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE']:
        new_value = 'Nordeste'
    elif value in ['AC', 'RO', 'AM', 'AP', 'PA', 'RR', 'TO']:
        new_value = 'Norte'
    else:
        new_value = 'SP'
    return new_value

df_train['internal_feature_11'] = df_train['internal_feature_11'].apply(lambda x: get_regiao(x))
df_val['internal_feature_11'] = df_val['internal_feature_11'].apply(lambda x: get_regiao(x))

In [5]:
trans_dict = {}

for feature in ['internal_feature_0', 'internal_feature_4']:

    temp = df_train[feature] \
            .value_counts(normalize = True) \
            .sort_values(ascending = False) \
            .reset_index() \
            .rename(columns = {'index': feature, feature: 'values'})

    temp['values'] = temp.apply(lambda x: x[0] if x[1] > 0.01 else 'Others', axis = 1)

    temp_dict = {}
    for line in temp[feature]:        
        temp_dict[line] = temp.loc[temp[feature] == line, 'values'].values[0]
    
    trans_dict[feature] = temp_dict
    
for feature in trans_dict.keys():
    df_train[feature] = df_train[feature].apply(lambda x: trans_dict[feature][x])

In [6]:
# internal_feature_5
df_train['internal_feature_5'].unique()

array(['PCRE', 'PDIV', 'WEDD', 'PDEB', 'TRAV', 'EDUC', 'BUSI', 'FURN',
       'VEIC', 'IMOB', 'DEBT', 'ELET', 'IEMP', 'MEDI', 'PORT', 'PEMP',
       'ESTH'], dtype=object)

In [7]:
# internal_feature_9

def get_status(value):
    if value in ['DIVORCED', 'SEPARATED']:
        new_value = 'DIVORCED'
    elif value in ['MARRIED', 'STABLE_UNION']:
        new_value = 'MARRIED'
    else:
        new_value = 'SINGLE'
    return new_value

df_train['internal_feature_9'] = df_train['internal_feature_9'].apply(lambda x: get_status(x))
df_val['internal_feature_9'] = df_val['internal_feature_9'].apply(lambda x: get_status(x))

In [8]:
bureau_cat = ['bureau_feature_16', 'bureau_feature_17', 'bureau_feature_18', 
              'bureau_feature_19', 'bureau_feature_20', 'bureau_feature_21']

for feature in bureau_cat:
    df_train[feature] = df_train[feature].apply(lambda x: 'NAO_CONSTAM_OCORRENCIAS' if x == 'Not_found' else x)
    df_val[feature] = df_val[feature].apply(lambda x: 'NAO_CONSTAM_OCORRENCIAS' if x == 'Not_found' else x)

Treatment for numerical columns